# Load drive, functions, libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Libraries

In [ ]:
!pip install -q keras

In [ ]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization, Bidirectional, TimeDistributed
#from tensorflow.python.keras.layers import Dense, Flatten, Permute, Reshape, Activation, add, MaxPooling1D, concatenate, GlobalAveragePooling2D, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, Flatten, Permute, Reshape, Activation, add, GlobalMaxPooling1D, concatenate, GlobalAveragePooling2D, GlobalAveragePooling1D, Dropout, Conv1D, Conv2D, MaxPooling2D, Input, Multiply
#from tensorflow.python.keras.layers import Dropout, Conv1D, Conv2D, MaxPooling2D, MaxPooling3D, AveragePooling2D, Input, Multiply
#from tensorflow.python.keras.callbacks import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import SyncBatchNormalization #Without tf.distribute strategy, this layer behaves as a regular tf.keras.layers.BatchNormalization layer....used to save model


In [ ]:
#import numpy as np

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score, roc_curve, auc, ConfusionMatrixDisplay
from sklearn.metrics import matthews_corrcoef, roc_auc_score, average_precision_score
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler

import pandas as pd
import string
import matplotlib.pyplot as plt
import scipy.io as sio
import numpy as np

import math
#import seaborn as sns

#sns.set(font_scale = 1.7)


In [ ]:
def pa1(preds, test_sc_labelcat,numClasses): #source counting accuracy metric function. Input predicted labels, true labels, and number of classes
  if len(preds.shape)==1:
    yhat = preds
  else:
    yhat = np.argmax(preds,axis=1) #predicted labels
  if len(test_sc_labelcat.shape)==1:
    y = test_sc_labelcat
  else:
    y = np.argmax(test_sc_labelcat,axis=1)  #true labels

  score=0 #for  pa1
  paw_score = 0
  values, counts = np.unique(y, return_counts=True) #'counts' to get the number of samples per class
  perclass_acc_plusminus1 = np.zeros(numClasses) #variable to store per class accuracies plus/minus 1
  perclass_acc = np.zeros(len(values))

  cm = confusion_matrix(y,yhat)
  print(f'Average Accuracy: {np.trace(cm)/len(y)*100}%')
  for i in range(len(cm)):
    print(f'Accuracy per class source count {i+1}: {cm[i,i]/np.sum(cm[i,:])*100}%')

  print('\n Plus/minus 1 stats:')
  for i in range(len(y)):
    plus1 = yhat[i]+1
    minus1 = yhat[i]-1
    if yhat[i]==y[i]:
      perclass_acc[y[i]]+=1
      score=score+1
      paw_score+=1
      perclass_acc_plusminus1[y[i]]+=1
    if plus1==y[i]:
      score=score+1
      paw_score = paw_score+0.5
      perclass_acc_plusminus1[y[i]]+=1
    if minus1==y[i]:
      score=score+1
      paw_score = paw_score+0.5
      perclass_acc_plusminus1[y[i]]+=1
  accuracy_plusminus1 = score/len(y) *100
  print(f'Accuracy plus/minus 1 source counted is: {accuracy_plusminus1}')
  print('Per class accuracy plus/minus 1 source counted.')
  for i in range(len(perclass_acc_plusminus1)):
    print(f' Class Source count {i+1}: {perclass_acc_plusminus1[i]/counts[i]*100}')
  print('\n----- PAW1 -----')
  print(f'Average PAW1: {paw_score/len(y) *100}')
  paw = (perclass_acc + perclass_acc_plusminus1)/2
  for j in range(len(paw)):
    print(f'PAW1 Class Source count {j+1}: {paw[j]/counts[j]*100}')
  return accuracy_plusminus1, perclass_acc_plusminus1/counts*100


# NN visualization
'''
from keras.utils import plot_model
plot_model(model, 'model.png')
'''
def model_plot(history):
  import matplotlib.pyplot as plt

  # Plot training & validation accuracy values
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Training', 'Validation'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Training', 'Validation'], loc='upper left')
  plt.show()

def plot_result(item):
  '''
  input: 'item' as the name for the metric to plot i.e. 'loss' or 'accuracy'
  '''
  plt.plot(history.history[item], label=item)
  plt.plot(history.history["val_" + item], label="val_" + item)
  plt.xlabel("Epochs")
  plt.ylabel(item)
  plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
  plt.legend()
  plt.grid()
  plt.show()

## Test set accuracy and confusion
def results(preds,test_labelcat,class_names):
  #input the predictions as one hot output predictions, true labels as one hot categorical
  if len(preds.shape)==1:
    yhat = preds
  else:
    yhat = np.argmax(preds,axis=1) #predicted labels
  if len(test_labelcat.shape)==1:
    y = test_labelcat
  else:
    y = np.argmax(test_labelcat,axis=1)  #true labels

  #y = np.argmax(test_labelcat,axis=1) #true labels
  #yhat = np.argmax(preds,axis=1) #predicted labels
  print(accuracy_score(y,yhat))
  #print(confusion_matrix(y,yhat))
  print(classification_report(y, yhat, target_names=class_names))
  cm = confusion_matrix(y,yhat)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
  disp.plot()
  plt.show()

  print('\n------ MCC -----')
  print(matthews_corrcoef(y, yhat))

  print('\n------ F1-score ------')
  print(f1_score(y,yhat, average='micro')) #micro weighing; can use 'macro' or 'weighted'

  print('\n------ AUC -------')
  print(roc_auc_score(test_labelcat,preds,average='micro'))

  from sklearn.metrics import RocCurveDisplay

  RocCurveDisplay.from_predictions(test_labelcat.ravel(), preds.ravel())
  plt.show()


def mse_mae_(test_sc_labelcat,preds,cmin):
  # categorical true labels, model predictions, 'cmin': min value fo the classes
  from sklearn.metrics import mean_squared_error, mean_absolute_error
  y = np.argmax(test_sc_labelcat,axis=1)  #true labels
  #preds = modelB.predict(test)
  yhat = np.argmax(preds,axis=1) #predicted labels
  numClasses = len(np.unique(y))
  print('Overall error metrics')
  print('MSE', mean_squared_error(y,yhat))
  print('MAE', mean_absolute_error(y,yhat))  #MAE reported in visual counting paper along with the OBO metric
  print('\n')
  for i in range(numClasses):
    y_true = y[np.where(y==i)]  #getting labels for specific class
    y_pred = yhat[np.where(y==i)] #getting corresponding predictions for specific class
    print('Class', i+cmin)
    print('MSE', mean_squared_error(y_true,y_pred))
    print('MAE', mean_absolute_error(y_true,y_pred))

def conf_nice(fine_preds, test_sc_labelcat, numClasses): #better version of above function
  y = np.argmax(test_sc_labelcat,axis=1)  #true labels
  yhat = np.argmax(fine_preds,axis=1) #predicted labels
  score=0
  #numClasses=12
  values, counts = np.unique(y, return_counts=True) #'counts' to get the number of samples per class
  perclass_acc_plusminus1 = np.zeros(numClasses) #variable to store per class accuracies plus/minus 1
  labels_range = [*range(1,numClasses+1)] #getting a list of the number of possible source counts
  disp = ConfusionMatrixDisplay.from_predictions(y+1, yhat+1, labels = labels_range,
                                                 normalize='true',
                                                 values_format='.1') #give the true labels and predicted labels. '+1' so its the correct range of source count problem where counting doesn't start at 0
  fig = disp.figure_
  fig.set_figwidth(10)
  fig.set_figheight(10)
  #plt.figure(figsize=(10,10))
  #disp.plot()
  plt.show()

def get_metrics(y_test,preds, class_names):
  # function to get sklearn metrics: classification report, confusion matrix, fscore, MCC, ROC-AUC
  # input: labels as numeric, and model predictions as numeric
  # class_names: list of the labels in sequence order

  from sklearn.metrics import matthews_corrcoef, f1_score, roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
  from sklearn.preprocessing import LabelBinarizer

  label_binarizer = LabelBinarizer().fit(y_test)  #needed for roc, auc. labels in shape of (num of samples, num of classes)
  y_onehot = label_binarizer.transform(y_test)
  preds_onehot = label_binarizer.transform(preds)

  print(classification_report(y_test, preds, target_names=class_names))
  cm = confusion_matrix(y_test,preds)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
  disp.plot()
  plt.show()


  print('\n------ MCC -----')
  print(matthews_corrcoef(y_test, preds))

  print('\n------ F1-score ------')
  print(f1_score(y_test,preds)) #micro weighing; can use 'macro' or 'weighted'

  print('\n------ AUC -------')
  print(roc_auc_score(y_onehot,preds_onehot))

def calculate_overall_lwlrap_sklearn(truth, scores):
  """Calculate the overall lwlrap using sklearn.metrics.lrap."""
  import sklearn.metrics
  # sklearn doesn't correctly apply weighting to samples with no labels, so just skip them.
  sample_weight = np.sum(truth > 0, axis=1)
  nonzero_weight_sample_indices = np.flatnonzero(sample_weight > 0)
  overall_lwlrap = sklearn.metrics.label_ranking_average_precision_score(
      truth[nonzero_weight_sample_indices, :] > 0,
      scores[nonzero_weight_sample_indices, :],
      sample_weight=sample_weight[nonzero_weight_sample_indices])
  return overall_lwlrap

## Function to make HCF

In [ ]:
from scipy import signal
from scipy.stats import kurtosis
import librosa
#https://librosa.org/doc/latest/feature.html
#Function: from a list of audio files extract acoustic features using Librosa

def acoustics(train_audio):
  #root mean square: takes raw audio signal, makes overlapping frames, and computes sqrt(power) of each frame as RMS
  rms_list = np.array([librosa.feature.rms(y=audio, frame_length=2048, hop_length=1024)
                     for audio in train_audio])
#spectral centroid: given audio signal, mag spectrogram computed.
# Each frame of a magnitude spectrogram is normalized and treated as a distribution over frequency bins,
# from which the mean (centroid) isextracted per frame and returned as a vector.
  cs_list = np.array([librosa.feature.spectral_centroid(y=audio, n_fft=2048, hop_length=1024)
                    for audio in train_audio])
#spectral bandwidth
  bw_list = np.array([librosa.feature.spectral_bandwidth(y=audio, n_fft=2048, hop_length=1024)
                    for audio in train_audio])
#spectral flatness
  fl_list = np.array([librosa.feature.spectral_flatness(y=audio, n_fft=2048, hop_length=1024)
                    for audio in train_audio])
#spectral rolloff
  roll_list = np.array([librosa.feature.spectral_rolloff(y=audio, n_fft=2048, hop_length=1024)
                      for audio in train_audio])


#to fix the shape?
  rms_list = rms_list[:,0,:]
  cs_list = cs_list[:,0,:]
  bw_list = bw_list[:,0,:]
  fl_list = fl_list[:,0,:]
  roll_list = roll_list[:,0,:]

  return rms_list,cs_list,bw_list,fl_list,roll_list


#additional features from the STFT, called in 'spec_features'
def get0_power(a, threshold):
    mint = -80 ## minimum threshold
    if threshold == mint:
        idx = np.where(a == threshold)[0]
    else:
        idx = np.where(a <= threshold)[0]
    if len(idx) > 1:
        idx = idx[0]
    elif len(idx) == 0:
        idx = 128 #len(a)
    return(idx)


#mel spectrogram calculation features
def spec_features(audio_list): #input list of all audio signals

  stft_list = np.array([librosa.feature.melspectrogram(y=audio, sr=44100,
                                                     n_mels=64, fmax=22050, #fmax=sr/2
                                                     n_fft=2048,
                                                     hop_length=1024)
                          for audio in audio_list])
  print('stft done')
  #convert melspec to dB scale
  stft_list_dB = np.array([librosa.power_to_db(stft_list[i,:,:], ref=np.max)
                          for i in range(stft_list.shape[0])])
  print('stft db done')

  ## median power vs time
  medp_time = np.array([np.apply_along_axis(np.median,  0, stft_list_dB[i,:,:])
                      for i in range(stft_list.shape[0])])
  ## mean power vs. time
  meanp_time = np.array([np.apply_along_axis(np.mean,  0, stft_list_dB[i,:,:])
                       for i in range(stft_list.shape[0])])
  print('power vs time done')

  ## time to power of -80
  tp80 = np.array([np.apply_along_axis(get0_power,  1, stft_list_dB[i,:,:], -80).flatten()
                 for i in range(stft_list.shape[0])])
  ## time to power of -75
  tp75 = np.array([np.apply_along_axis(get0_power,  1, stft_list_dB[i,:,:], -75).flatten()
                 for i in range(stft_list.shape[0])])
  ## time to power of -70
  tp70 = np.array([np.apply_along_axis(get0_power,  1, stft_list_dB[i,:,:], -70).flatten()
                 for i in range(stft_list.shape[0])])
  print('time to 80,75, 70 done')


  #### wavelet features #### looking at widths of 1, 5, 10, 25
  width = 1
  addwavelets_1 = np.zeros((7, stft_list_dB.shape[0], stft_list_dB.shape[1]))

  for j in range(stft_list_dB.shape[0]):
    wavetransform = np.array([signal.cwt(stft_list_dB[j, i, :], signal.ricker,[width])[0]
                              for i in range(stft_list_dB.shape[1])])
    addwavelets_1[0, j,:] = np.mean(wavetransform, axis = 1) #mean wavelet
    addwavelets_1[1, j,:] = np.median(wavetransform, axis = 1) #median wavelet
    addwavelets_1[2, j,:] = np.std(wavetransform, axis = 1) #standard dev
    addwavelets_1[3, j,:] = np.var(wavetransform, axis = 1) #variance
    addwavelets_1[4, j,:] = kurtosis(wavetransform, axis = 1) #kurtosis
    addwavelets_1[5, j,:] = np.quantile(wavetransform, 0.25, axis = 1) #25th quantile
    addwavelets_1[6, j,:] = np.quantile(wavetransform, 0.75, axis = 1) #75th quantile...need to explain significance of these
    if j % 1000 == 0:
        print(j)
  print('wavelet 1 done')

  width = 5
  addwavelets_5 = np.zeros((7, stft_list_dB.shape[0], stft_list_dB.shape[1]))

  for j in range(stft_list_dB.shape[0]):
    wavetransform = np.array([signal.cwt(stft_list_dB[j, i, :], signal.ricker,[width])[0]
                              for i in range(stft_list_dB.shape[1])])
    addwavelets_5[0, j,:] = np.mean(wavetransform, axis = 1) #mean wavelet
    addwavelets_5[1, j,:] = np.median(wavetransform, axis = 1) #median wavelet
    addwavelets_5[2, j,:] = np.std(wavetransform, axis = 1) #standard dev
    addwavelets_5[3, j,:] = np.var(wavetransform, axis = 1) #variance
    addwavelets_5[4, j,:] = kurtosis(wavetransform, axis = 1) #kurtosis
    addwavelets_5[5, j,:] = np.quantile(wavetransform, 0.25, axis = 1) #25th quantile
    addwavelets_5[6, j,:] = np.quantile(wavetransform, 0.75, axis = 1) #75th quantile...need to explain significance of these
    if j % 1000 == 0:
        print(j)
  print('wavelet 5 done')


  width = 10
  addwavelets_10 = np.zeros((7, stft_list_dB.shape[0], stft_list_dB.shape[1]))

  for j in range(stft_list_dB.shape[0]):
    wavetransform = np.array([signal.cwt(stft_list_dB[j, i, :], signal.ricker,[width])[0]
                              for i in range(stft_list_dB.shape[1])])
    addwavelets_10[0, j,:] = np.mean(wavetransform, axis = 1) #mean wavelet
    addwavelets_10[1, j,:] = np.median(wavetransform, axis = 1) #median wavelet
    addwavelets_10[2, j,:] = np.std(wavetransform, axis = 1) #standard dev
    addwavelets_10[3, j,:] = np.var(wavetransform, axis = 1) #variance
    addwavelets_10[4, j,:] = kurtosis(wavetransform, axis = 1) #kurtosis
    addwavelets_10[5, j,:] = np.quantile(wavetransform, 0.25, axis = 1) #25th quantile
    addwavelets_10[6, j,:] = np.quantile(wavetransform, 0.75, axis = 1) #75th quantile...need to explain significance of these
    if j % 1000 == 0:
        print(j)
  print('wavelet 10 done')

  width = 25
  addwavelets_25 = np.zeros((7, stft_list_dB.shape[0], stft_list_dB.shape[1]))

  for j in range(stft_list_dB.shape[0]):
    wavetransform = np.array([signal.cwt(stft_list_dB[j, i, :], signal.ricker,[width])[0]
                              for i in range(stft_list_dB.shape[1])])
    addwavelets_25[0, j,:] = np.mean(wavetransform, axis = 1) #mean wavelet
    addwavelets_25[1, j,:] = np.median(wavetransform, axis = 1) #median wavelet
    addwavelets_25[2, j,:] = np.std(wavetransform, axis = 1) #standard dev
    addwavelets_25[3, j,:] = np.var(wavetransform, axis = 1) #variance
    addwavelets_25[4, j,:] = kurtosis(wavetransform, axis = 1) #kurtosis
    addwavelets_25[5, j,:] = np.quantile(wavetransform, 0.25, axis = 1) #25th quantile
    addwavelets_25[6, j,:] = np.quantile(wavetransform, 0.75, axis = 1) #75th quantile...need to explain significance of these
    if j % 1000 == 0:
        print(j)

  return stft_list, stft_list_dB, medp_time, meanp_time, tp80, tp75, tp70, addwavelets_1, addwavelets_5, addwavelets_10, addwavelets_25






In [ ]:
# additional acoustic features;
# treat mfccs just like the wavelet based features (can take stats from it like mean, std, kurt,)
import numpy as np
import librosa
import pandas as pd
###################################
import sys
eps = sys.float_info.epsilon

def additional_features(audio, split, split_scene,frame_len, hop_len):
  sr = 44100
  frame_len = 2048
  hop_len = 1024
  flag = 0
  mfcc=[]
  mfcc_d=[]
  mfcc_dd=[]

  #for count,y in enumerate(audio): #list of raw audio signals
  for i in range(len(audio)): #dataframe to get list of audio file names
    audiofile = df['Filename'][i].split("/") #get files in order, split string for audio file only not path
    f = audiofile[7]
    filename = f'/content/drive/My Drive/SARdBScene/{split}/{split_scene}/{f}'
    y,sr = librosa.load(filename,sr=44100)


    S = np.abs(librosa.stft(y,n_fft=frame_len, hop_length=hop_len)) #gets the STFT mag or spectrogram (rows are freq. axis=0)
    print(i)

    m = librosa.feature.mfcc(y=y, sr=sr,n_fft=frame_len, hop_length=hop_len, n_mfcc=13)
    mfcc.append(m)
    mfcc_d.append(librosa.feature.delta(m))
    mfcc_dd.append(librosa.feature.delta(librosa.feature.delta(m)))

    if flag==0: #only for the first data sample, then we concat the features for one storage variable
      # zero crossing rate
      zcr = librosa.feature.zero_crossing_rate(y,frame_length = frame_len, hop_length=hop_len)

      #
      # 1) compute probabilty distribution of spectrogram
      p = S / (np.sum(S,axis=0)+eps)
      # 2) calculate entropy of each time frame
      H = -np.sum(p * np.log2(p+eps),axis=0).reshape((1,431))

    elif flag>0:
      zcr = np.concatenate((zcr,librosa.feature.zero_crossing_rate(y,frame_length = frame_len, hop_length=hop_len)),axis=0)
      m = librosa.feature.mfcc(y=y, sr=sr,n_fft=frame_len, hop_length=hop_len, n_mfcc=13)
      #
      p = S / (np.sum(S,axis=0)+eps)
      H = np.concatenate((H, -np.sum(p*np.log2(p+eps),axis=0).reshape((1,431))),axis=0)

    flag = 10

  mfcc = np.reshape(np.array([mfcc]),(len(audio),13,431))  #make as array type and reshape to remove additional axis
  mfcc_d = np.reshape(np.array([mfcc_d]),(len(audio),13,431))
  mfcc_dd = np.reshape(np.array([mfcc_dd]),(len(audio),13,431))

  return zcr, mfcc, mfcc_d, mfcc_dd, H



## CKA Function

In [ ]:
#https://colab.research.google.com/github/google-research/google-research/blob/master/representation_similarity/Demo.ipynb#scrollTo=MkucRi3yn7UJ

import math
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
def gram_linear(x):
  """Compute Gram (kernel) matrix for a linear kernel.

  Args:
    x: A num_examples x num_features matrix of features.

  Returns:
    A num_examples x num_examples Gram matrix of examples.
  """
  return x.dot(x.T)


def gram_rbf(x, threshold=1.0):
  """Compute Gram (kernel) matrix for an RBF kernel.

  Args:
    x: A num_examples x num_features matrix of features.
    threshold: Fraction of median Euclidean distance to use as RBF kernel
      bandwidth. (This is the heuristic we use in the paper. There are other
      possible ways to set the bandwidth; we didn't try them.)

  Returns:
    A num_examples x num_examples Gram matrix of examples.
  """
  dot_products = x.dot(x.T)
  sq_norms = np.diag(dot_products)
  sq_distances = -2 * dot_products + sq_norms[:, None] + sq_norms[None, :]
  sq_median_distance = np.median(sq_distances)
  return np.exp(-sq_distances / (2 * threshold ** 2 * sq_median_distance))


def center_gram(gram, unbiased=False):
  """Center a symmetric Gram matrix.

  This is equvialent to centering the (possibly infinite-dimensional) features
  induced by the kernel before computing the Gram matrix.

  Args:
    gram: A num_examples x num_examples symmetric matrix.
    unbiased: Whether to adjust the Gram matrix in order to compute an unbiased
      estimate of HSIC. Note that this estimator may be negative.

  Returns:
    A symmetric matrix with centered columns and rows.
  """
  if not np.allclose(gram, gram.T):
    raise ValueError('Input must be a symmetric matrix.')
  gram = gram.copy()

  if unbiased:
    # This formulation of the U-statistic, from Szekely, G. J., & Rizzo, M.
    # L. (2014). Partial distance correlation with methods for dissimilarities.
    # The Annals of Statistics, 42(6), 2382-2412, seems to be more numerically
    # stable than the alternative from Song et al. (2007).
    n = gram.shape[0]
    np.fill_diagonal(gram, 0)
    means = np.sum(gram, 0, dtype=np.float64) / (n - 2)
    means -= np.sum(means) / (2 * (n - 1))
    gram -= means[:, None]
    gram -= means[None, :]
    np.fill_diagonal(gram, 0)
  else:
    means = np.mean(gram, 0, dtype=np.float64)
    means -= np.mean(means) / 2
    gram -= means[:, None]
    gram -= means[None, :]

  return gram


def cka(gram_x, gram_y, debiased=False):
  """Compute CKA.

  Args:
    gram_x: A num_examples x num_examples Gram matrix.
    gram_y: A num_examples x num_examples Gram matrix.
    debiased: Use unbiased estimator of HSIC. CKA may still be biased.

  Returns:
    The value of CKA between X and Y.
  """
  gram_x = center_gram(gram_x, unbiased=debiased)
  gram_y = center_gram(gram_y, unbiased=debiased)

  # Note: To obtain HSIC, this should be divided by (n-1)**2 (biased variant) or
  # n*(n-3) (unbiased variant), but this cancels for CKA.
  scaled_hsic = gram_x.ravel().dot(gram_y.ravel())

  normalization_x = np.linalg.norm(gram_x)
  normalization_y = np.linalg.norm(gram_y)
  return scaled_hsic / (normalization_x * normalization_y)


def _debiased_dot_product_similarity_helper(
    xty, sum_squared_rows_x, sum_squared_rows_y, squared_norm_x, squared_norm_y,
    n):
  """Helper for computing debiased dot product similarity (i.e. linear HSIC)."""
  # This formula can be derived by manipulating the unbiased estimator from
  # Song et al. (2007).
  return (
      xty - n / (n - 2.) * sum_squared_rows_x.dot(sum_squared_rows_y)
      + squared_norm_x * squared_norm_y / ((n - 1) * (n - 2)))


def feature_space_linear_cka(features_x, features_y, debiased=False):
  """Compute CKA with a linear kernel, in feature space.

  This is typically faster than computing the Gram matrix when there are fewer
  features than examples.

  Args:
    features_x: A num_examples x num_features matrix of features.
    features_y: A num_examples x num_features matrix of features.
    debiased: Use unbiased estimator of dot product similarity. CKA may still be
      biased. Note that this estimator may be negative.

  Returns:
    The value of CKA between X and Y.
  """
  features_x = features_x - np.mean(features_x, 0, keepdims=True)
  features_y = features_y - np.mean(features_y, 0, keepdims=True)

  dot_product_similarity = np.linalg.norm(features_x.T.dot(features_y)) ** 2
  normalization_x = np.linalg.norm(features_x.T.dot(features_x))
  normalization_y = np.linalg.norm(features_y.T.dot(features_y))

  if debiased:
    n = features_x.shape[0]
    # Equivalent to np.sum(features_x ** 2, 1) but avoids an intermediate array.
    sum_squared_rows_x = np.einsum('ij,ij->i', features_x, features_x)
    sum_squared_rows_y = np.einsum('ij,ij->i', features_y, features_y)
    squared_norm_x = np.sum(sum_squared_rows_x)
    squared_norm_y = np.sum(sum_squared_rows_y)

    dot_product_similarity = _debiased_dot_product_similarity_helper(
        dot_product_similarity, sum_squared_rows_x, sum_squared_rows_y,
        squared_norm_x, squared_norm_y, n)
    normalization_x = np.sqrt(_debiased_dot_product_similarity_helper(
        normalization_x ** 2, sum_squared_rows_x, sum_squared_rows_x,
        squared_norm_x, squared_norm_x, n))
    normalization_y = np.sqrt(_debiased_dot_product_similarity_helper(
        normalization_y ** 2, sum_squared_rows_y, sum_squared_rows_y,
        squared_norm_y, squared_norm_y, n))

  return dot_product_similarity / (normalization_x * normalization_y)

Linear CKA can be computed either based on dot products between examples or dot products between features:
$$\langle\text{vec}(XX^\text{T}),\text{vec}(YY^\text{T})\rangle = ||Y^\text{T}X||_\text{F}^2$$
The formulation based on similarities between features (right-hand side) is faster than the formulation based on similarities between similarities between examples (left-hand side) when the number of examples exceeds the number of features. We provide both formulations here and demonstrate that they are equvialent.

# Deep Features

**YAMNet**


In [ ]:
def ensure_sample_rate(original_sample_rate, waveform,
                       desired_sample_rate=16000):
  """Resample waveform if required. YAMNet needs 16kHz audio and normalized to [-1,1]"""
  if original_sample_rate != desired_sample_rate:
    desired_length = int(round(float(len(waveform)) /
                               original_sample_rate * desired_sample_rate))
    waveform = scipy.signal.resample(waveform, desired_length)

    waveform = waveform / tf.int16.max #normalize to [-1,+1]
  return desired_sample_rate, waveform

In [ ]:
# load yamnet model
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [ ]:
#%% Read csv file as dataframe

"""Change variable 'split_scene' and 'split' according to the data to be used for feature extraction.
Repeat cell as needed to get dataset features"""

split_scene = 'test_nature'
split = 'test'

df = pd.read_csv(f'/content/drive/My Drive/SARdBScene_files/SARdBScene_annotations/{split_scene}.csv')

#source_count_labels = df['Source Count'].to_numpy() #already saved source count labels

embedding_list=[]
files = []
for i in range(len(df)):
    print(i, '/', len(df))
    audiofile = df['Filename'][i].split("/") #get files in order, split string for audio file only not path
    f = audiofile[7]
    filename = f'/content/drive/My Drive/SARdBScene/{split}/{split_scene}/{f}'

    sample_rate1, testing_wav_data = wavfile.read(filename, 'rb')
    sample_rate, testing_wav_data = ensure_sample_rate(sample_rate1, testing_wav_data)
    score, embedding, spec = yamnet_model(testing_wav_data)

    embedding_list.append(embedding)
    files.append(filename) #check

embedding_list_np = np.array(embedding_list)
np.save(f'/content/drive/My Drive/Colab Notebooks/SARdBScene_features/{split_scene}_yamnet.npy',embedding_list_np)


**OpenL3**

In [ ]:
import openl3
import soundfile as sf
import pandas as pd
import numpy as np

In [ ]:
#%% Read csv file as dataframe
"""Change variable 'split_scene' and 'split' according to the data to be used for feature extraction.
Repeat cell as needed to get dataset features"""

split_scene = 'valid_nature'
split = 'valid'

df = pd.read_csv(f'/content/drive/My Drive/SARdBScene_files/SARdBScene_annotations/{split_scene}.csv')

#source_count_labels = df['Source Count'].to_numpy() #already saved source count labels

model = openl3.models.load_audio_embedding_model(input_repr="mel256", content_type="env",embedding_size=512)

embedding_list=[]
files = []
for i in range(len(df)):
    print(i+1, '/', len(df))
    audiofile = df['Filename'][i].split("/") #get files in order, split string for audio file only not path
    filename = audiofile[7]
    y,sr = sf.read(f'/content/drive/My Drive/SARdBScene/{split}/{split_scene}/{filename}')#sr=44100
    embedding, t2 = openl3.get_audio_embedding(y, sr, model=model,verbose=0)

    embedding_list.append(embedding)
    files.append(filename) #check

embedding_list_np = np.array(embedding_list)
np.save(f'/content/drive/My Drive/ColabNotebooks/SARdBScene_features/{split_scene}_openL3.npy',embedding_list_np)


# Load Data

After hand-crafted features and deep features are saved. Load only the data needed for analysis (shown loads from 4 subsets of sardbscene)

In [ ]:
train_zcr = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/zcr_train_office.npy"),
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/zcr_train_urban.npy"),
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/zcr_train_home.npy"),
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/zcr_train_nature.npy")))

scaler = StandardScaler(with_mean=True, with_std=True).fit(train_zcr)
train_zcr = scaler.transform(train_zcr)
print(train_zcr.shape)

#entropy
train_ent =np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/spectral_entropy_train_office.npy"),
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/spectral_entropy_train_urban.npy"),
                           np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/spectral_entropy_train_home.npy"),
                           np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/spectral_entropy_train_nature.npy")))
scaler = StandardScaler(with_mean=True, with_std=True).fit(train_ent)
train_ent = scaler.transform(train_ent)
print(train_ent.shape)

#cepstral
#atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_office.npz")
#mfcc = atr["arr_0"]
#mfcc_d = atr["arr_1"]
#mfcc_dd = atr["arr_2"]
mfcc = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_office.npz")["arr_0"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_urban.npz")["arr_0"],
                           np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_home.npz")["arr_0"],
                           np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_nature.npz")["arr_0"]))

mfcc_d = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_office.npz")["arr_1"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_urban.npz")["arr_1"],
                           np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_home.npz")["arr_1"],
                           np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_nature.npz")["arr_1"]))

mfcc_dd = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_office.npz")["arr_2"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_urban.npz")["arr_2"],
                           np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_home.npz")["arr_2"],
                           np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_nature.npz")["arr_2"]))

mfcc = np.reshape(mfcc,(len(mfcc),mfcc.shape[1]*mfcc.shape[2]))
scaler = StandardScaler(with_mean=True, with_std=True).fit(mfcc)
mfcc = scaler.transform(mfcc)
print(mfcc.shape)

mfcc_d = np.reshape(mfcc_d,(len(mfcc_d),mfcc_d.shape[1]*mfcc_d.shape[2]))
scaler = StandardScaler(with_mean=True, with_std=True).fit(mfcc_d)
mfcc_d = scaler.transform(mfcc_d)
print(mfcc_d.shape)

mfcc_dd = np.reshape(mfcc_dd,(len(mfcc_dd), mfcc_dd.shape[1]*mfcc_dd.shape[2]))
scaler = StandardScaler(with_mean=True, with_std=True).fit(mfcc_dd)
mfcc_dd = scaler.transform(mfcc_dd)
print(mfcc_dd.shape)

In [ ]:
cka(gram_linear(mfcc), gram_linear(mfcc_dd))

In [ ]:
cka(gram_linear(mfcc), gram_linear(mfcc))

In [ ]:
#atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_nature.npz")
# npz file saved with rms, cs, bw, fl, roll
# acoustic features are 1x431 dimensional
train_rms = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_office.npz")["arr_0"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustics_train_urban.npz")["arr_0"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_home.npz")["arr_0"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustics_train_nature.npz")["arr_0"]))

train_cs = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_office.npz")["arr_1"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustics_train_urban.npz")["arr_1"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_home.npz")["arr_1"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustics_train_nature.npz")["arr_1"]))

train_bw = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_office.npz")["arr_2"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustics_train_urban.npz")["arr_2"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_home.npz")["arr_2"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustics_train_nature.npz")["arr_2"]))

train_fl = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_office.npz")["arr_3"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustics_train_urban.npz")["arr_3"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_home.npz")["arr_3"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustics_train_nature.npz")["arr_3"]))

train_roll = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_office.npz")["arr_4"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustics_train_urban.npz")["arr_4"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_home.npz")["arr_4"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustics_train_nature.npz")["arr_4"]))


## Normalize all features
scaler = StandardScaler(with_mean=True, with_std=True).fit(train_rms)
train_rms = scaler.transform(train_rms)

scaler = StandardScaler(with_mean=True, with_std=True).fit(train_cs)
train_cs = scaler.transform(train_cs)

scaler = StandardScaler(with_mean=True, with_std=True).fit(train_bw)
train_bw = scaler.transform(train_bw)

scaler = StandardScaler(with_mean=True, with_std=True).fit(train_fl)
train_fl = scaler.transform(train_fl)

scaler = StandardScaler(with_mean=True, with_std=True).fit(train_roll)
train_roll = scaler.transform(train_roll)

power_tr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_nature.npz")
# spectrogram features of median power vs time (medp_time), meanp_time, tp80, tp75, tp70
# power features are 1x431 dimensional, time to dB features are 1x64
train_medp = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_office.npz")["arr_0"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_urban.npz")["arr_0"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_home.npz")["arr_0"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_nature.npz")["arr_0"]))

train_meanp = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_office.npz")["arr_1"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_urban.npz")["arr_1"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_home.npz")["arr_1"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_nature.npz")["arr_1"]))

train_tp80 = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_office.npz")["arr_2"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_urban.npz")["arr_2"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_home.npz")["arr_2"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_nature.npz")["arr_2"]))

train_tp75 = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_office.npz")["arr_3"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_urban.npz")["arr_3"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_home.npz")["arr_3"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_nature.npz")["arr_3"]))

train_tp70 = np.concatenate((np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_office.npz")["arr_4"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_urban.npz")["arr_4"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_home.npz")["arr_4"],
                            np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/power_time_train_nature.npz")["arr_4"]))

## Normalize all features

scaler =  StandardScaler(with_mean=True, with_std=True).fit(train_medp)
train_medp = scaler.transform(train_medp)

scaler =  StandardScaler(with_mean=True, with_std=True).fit(train_meanp)
train_meanp = scaler.transform(train_meanp)

scaler = StandardScaler(with_mean=True, with_std=True).fit(train_tp80)
train_tp80 = scaler.transform(train_tp80)

scaler = StandardScaler(with_mean=True, with_std=True).fit(train_tp75)
train_tp75 = scaler.transform(train_tp75)

scaler = StandardScaler(with_mean=True, with_std=True).fit(train_tp70)
train_tp70 = scaler.transform(train_tp70)

#
# wavelets
wave_nature = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_nature.npy')
wave_office = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_office.npy')
wave_urban = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_urban.npy')
wave_home = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_home.npy')

wave1_tr = np.concatenate((wave_office, wave_urban, wave_home, wave_nature))
# wavelet width 1, features per row as mean, meadian, std, var, kurtosis, 25th quantile, 75th quantile
# wavelet features are all 1x64 dimensional vectors
scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[0,:,:])
train_wave1_mean = scaler.transform(wave1_tr[0,:,:])

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[1,:,:])
train_wave1_med = scaler.transform(wave1_tr[1,:,:])

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[2,:,:])
train_wave1_std = scaler.transform(wave1_tr[2,:,:])

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[3,:,:])
train_wave1_var = scaler.transform(wave1_tr[3,:,:])

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[4,:,:])
train_wave1_kur = scaler.transform(wave1_tr[4,:,:])

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[5,:,:])
train_wave1_q25 = scaler.transform(wave1_tr[5,:,:])

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[6,:,:])
train_wave1_q75 = scaler.transform(wave1_tr[6,:,:])

# CKA Similarity

Iteratively calculate linear CKA metric for each combination of hand-crafted feature and deep feature set. Shown for using the `Nature' subset of SARdBScene.

In [ ]:
## Add random noise for baseline
train_random = np.random.normal(0, 1,
                                size = train_rms.shape[0]*train_rms.shape[1]).reshape(train_rms.shape)

# list of features for easier access of cka operation
spec_features = [train_zcr, train_ent, train_rms, train_cs, train_bw, train_fl, train_roll,
                 train_medp, train_meanp, train_tp80, train_tp75, train_tp70,
                 mfcc, mfcc_d, mfcc_dd,
                 train_wave1_mean, train_wave1_med, train_wave1_std, train_wave1_var,
                 train_wave1_kur, train_wave1_q25, train_wave1_q75, train_random]

ckatrain = np.zeros((2, len(spec_features), 1))

# CKA for pretrain audio features and hand crafted acoustics (spec_features variable)

pp = '/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_'
arch_list = ['yamnet', 'openL3']
for k in range(1): ## inits
    for i in range(len(arch_list)):
        ## i - architecture
        feat = np.load(pp+arch_list[i]+'.npy')
        feat = np.mean(feat,axis=1)
        scaler = StandardScaler(with_mean=True, with_std=True).fit(feat)
        feat = scaler.transform(feat)
        ## Flatten and scale

        for j in range(len(spec_features)):
            ## j - hand-crafted feature
            ## Calculate linear CKA
            #ckatrain[i,j,k] = linear_CKA2(feat, spec_features[j])
            ckatrain[i,j,k] = cka(gram_linear(feat), gram_linear(spec_features[j]))

            print(i, j, k)

yticklabels = ['YAMNet', 'OpenL3']
xticklabels = ['ZCR', 'Spec. Entropy', 'RMS', 'Spec. Centroid', 'Spec. Bandwidth', 'Spec. Flatness', 'Spec. Rolloff',
               'Median Power', 'Mean Power', 'Time to -80dB',
               'Time to -75dB','Time to -70dB',
               'MFCC', 'MFCC-delta', 'MFCC-delta delta',
               'Mean Wavelet-1',
               'Median Wavelet-1', 'Std. Wavelet-1','Var. Wavelet-1','Kurtosis Wavelet-1',
               '25th Quantile Wavelet-1', '75th Quantile Wavelet-1','Random']
plt.figure(figsize = (30,10))
sns.heatmap(np.round(np.mean(ckatrain, axis = 2), 2),  vmin = 0, vmax = 1, annot=True,
           xticklabels = xticklabels, yticklabels = yticklabels)
plt.yticks(rotation=0)
plt.tight_layout()

In [ ]:
#differenet sized plot, no values displayed in cells
plt.figure(figsize = (30,5))
sns.heatmap(np.round(np.mean(ckatrain, axis = 2), 2),  vmin = 0, vmax = 1, annot=False,
           xticklabels = xticklabels, yticklabels = yticklabels)
plt.yticks(rotation=0)
plt.tight_layout()

In [ ]:
### VERTICAL PLOT
models = ['YAMNet', 'OpenL3']
feats = ['ZCR', 'Spec. Entropy', 'RMS', 'Spec. Centroid', 'Spec. Bandwidth', 'Spec. Flatness', 'Spec. Rolloff',
               'Median Power', 'Mean Power', 'Time to -80dB',
               'Time to -75dB','Time to -70dB',
               'MFCC', 'MFCC-delta', 'MFCC-delta delta',
               'Mean Wavelet-1',
               'Median Wavelet-1', 'Std. Wavelet-1','Var. Wavelet-1','Kurtosis Wavelet-1',
               '25th Quantile Wavelet-1', '75th Quantile Wavelet-1','Random']

plt.figure(figsize = (20,30))
sns.heatmap(ckatrain.T,  vmin = 0, vmax = 1, annot=False,
           xticklabels = models, yticklabels = feats)
plt.yticks(rotation=0)
plt.show()

## MFCC statistics

In [ ]:
from scipy.stats import kurtosis

atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/cepstral_train_nature.npz")
mfcc = atr["arr_0"]
mfcc_d = atr["arr_1"]
mfcc_dd = atr["arr_2"]
r = mfcc.shape[1]
c = mfcc.shape[2]

mfcc = np.reshape(mfcc,(len(mfcc),r*c))
mfcc_d = np.reshape(mfcc_d,(len(mfcc),r*c))
mfcc_dd = np.reshape(mfcc_dd,(len(mfcc),r*c))

scaler = StandardScaler(with_mean=True, with_std=True).fit(mfcc)
mfcc = scaler.transform(mfcc)

scaler = StandardScaler(with_mean=True, with_std=True).fit(mfcc_d)
mfcc_d = scaler.transform(mfcc_d)

scaler = StandardScaler(with_mean=True, with_std=True).fit(mfcc_dd)
mfcc_dd = scaler.transform(mfcc_dd)

mfcc = np.reshape(mfcc,(len(mfcc),r,c))
mfcc_d = np.reshape(mfcc_d,(len(mfcc),r,c))
mfcc_dd = np.reshape(mfcc_dd,(len(mfcc),r,c))

mfcc_meanf = np.mean(mfcc,axis=1) #mel band mean, so avg of the freq. for each time frame
mfcc_stdf = np.std(mfcc,axis=1)
mfcc_medf = np.median(mfcc,axis=1)
mfcc_varf = np.var(mfcc,axis=1)
mfcc_kurf = kurtosis(mfcc,axis=1)

mfcc_meant = np.mean(mfcc,axis=2)
mfcc_stdt = np.std(mfcc,axis=2)
mfcc_medt = np.median(mfcc,axis=2)
mfcc_vart = np.var(mfcc,axis=2)
mfcc_kurt = kurtosis(mfcc,axis=2)

#delta
mfcc_d_meanf = np.mean(mfcc_d,axis=1) #mel band mean, so avg of the freq. for each time frame
mfcc_d_stdf = np.std(mfcc_d,axis=1)
mfcc_d_medf = np.median(mfcc_d,axis=1)
mfcc_d_varf = np.var(mfcc_d,axis=1)
mfcc_d_kurf = kurtosis(mfcc_d,axis=1)

mfcc_d_meant = np.mean(mfcc_d,axis=2)
mfcc_d_stdt = np.std(mfcc_d,axis=2)
mfcc_d_medt = np.median(mfcc_d,axis=2)
mfcc_d_vart = np.var(mfcc_d,axis=2)
mfcc_d_kurt = kurtosis(mfcc_d,axis=2)

#double delta
mfcc_dd_meanf = np.mean(mfcc_dd,axis=1) #mel band mean, so avg of the freq. for each time frame
mfcc_dd_stdf = np.std(mfcc_dd,axis=1)
mfcc_dd_medf = np.median(mfcc_dd,axis=1)
mfcc_dd_varf = np.var(mfcc_dd,axis=1)
mfcc_dd_kurf = kurtosis(mfcc_dd,axis=1)

mfcc_dd_meant = np.mean(mfcc_dd,axis=2)
mfcc_dd_stdt = np.std(mfcc_dd,axis=2)
mfcc_dd_medt = np.median(mfcc_dd,axis=2)
mfcc_dd_vart = np.var(mfcc_dd,axis=2)
mfcc_dd_kurt = kurtosis(mfcc_dd,axis=2)

In [ ]:
print(mfcc_meanf.shape)
print(mfcc_meant.shape)

In [ ]:
meanf = np.concatenate((mfcc_meanf,mfcc_d_meanf,mfcc_dd_meanf),axis=1)
meant = np.concatenate((mfcc_meant,mfcc_d_meant,mfcc_dd_meant),axis=1)

stdf = np.concatenate((mfcc_stdf,mfcc_d_stdf,mfcc_dd_stdf),axis=1)
stdt = np.concatenate((mfcc_stdt,mfcc_d_stdt,mfcc_dd_stdt),axis=1)

medf = np.concatenate((mfcc_medf,mfcc_d_medf,mfcc_dd_medf),axis=1)
medt = np.concatenate((mfcc_medt,mfcc_d_medt,mfcc_dd_medt),axis=1)

varf = np.concatenate((mfcc_varf,mfcc_d_varf,mfcc_dd_varf),axis=1)
vart = np.concatenate((mfcc_vart,mfcc_d_vart,mfcc_dd_vart),axis=1)

kurf = np.concatenate((mfcc_kurf,mfcc_d_kurf,mfcc_dd_kurf),axis=1)
kurt = np.concatenate((mfcc_kurt,mfcc_d_kurt,mfcc_dd_kurt),axis=1)

In [ ]:
## Add random noise for baseline
#train_random = np.random.normal(0, 1,size = mfcc.shape[0]*train_wave1_mean.shape[1]).reshape(train_wave1_mean.shape)

# list of features for easier access of cka operation
spec_features = [mfcc_meanf, mfcc_stdf, mfcc_medf, mfcc_varf ,mfcc_kurf,
              mfcc_meant ,mfcc_stdt , mfcc_medt , mfcc_vart, mfcc_kurt,
              mfcc_d_meanf, mfcc_d_stdf, mfcc_d_medf, mfcc_d_varf, mfcc_d_kurf,
              mfcc_d_meant, mfcc_d_stdt, mfcc_d_medt, mfcc_d_vart,mfcc_d_kurt,
              mfcc_dd_meanf, mfcc_dd_stdf,mfcc_dd_medf , mfcc_dd_varf ,mfcc_dd_kurf,
              mfcc_dd_meant, mfcc_dd_stdt, mfcc_dd_medt ,mfcc_dd_vart,mfcc_dd_kurt]


ckatrain = np.zeros((2, len(spec_features), 1)) #7 feats per wavelet width x 4 widths = 28 +1 for random

# CKA for pretrain audio features and hand crafted acoustics (spec_features variable)

pp = '/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_'
arch_list = ['yamnet', 'openL3']
for k in range(1): ## inits
    for i in range(len(arch_list)):
        ## i - architecture
        feat = np.load(pp+arch_list[i]+'.npy')
        feat = np.reshape(feat, (len(feat), feat.shape[1]*feat.shape[2]))
        scaler = StandardScaler(with_mean=True, with_std=True).fit(feat)
        feat = scaler.transform(feat)
        ## Flatten and scale

        for j in range(len(spec_features)):
            ## j - hand-crafted feature
            ## Calculate linear CKA
            #ckatrain[i,j,k] = linear_CKA2(feat, spec_features[j])
            ckatrain[i,j,k] = cka(gram_linear(feat), gram_linear(spec_features[j]))

            print(i, j, k)

#plot
yticklabels = ['YAMNet', 'OpenL3']
xticklabels = ['mfcc_meanf', 'mfcc_stdf', 'mfcc_medf', 'mfcc_varf' , 'mfcc_kurf',
              'mfcc_meant' ,'mfcc_stdt' , 'mfcc_medt' , 'mfcc_vart', 'mfcc_kurt',
              'mfcc_d_meanf', 'mfcc_d_stdf', 'mfcc_d_medf', 'mfcc_d_varf', 'mfcc_d_kurf',
              'mfcc_d_meant', 'mfcc_d_stdt', 'mfcc_d_medt', 'mfcc_d_vart','mfcc_d_kurt',
              'mfcc_dd_meanf', 'mfcc_dd_stdf','mfcc_dd_medf' , 'mfcc_dd_varf' ,'mfcc_dd_kurf',
              'mfcc_dd_meant', 'mfcc_dd_stdt', 'mfcc_dd_medt' ,'mfcc_dd_vart','mfcc_dd_kurt']

plt.figure(figsize = (30,10))
sns.heatmap(np.round(np.mean(ckatrain, axis = 2), 2),  vmin = 0, vmax = 1, annot=True,
           xticklabels = xticklabels, yticklabels = yticklabels)
plt.yticks(rotation=0)
plt.tight_layout()

# Classification
perform classification of HCF with MLP classifiers

## 75th quantile wavelet

In [ ]:
## Load and organize data
wave1_tr = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_nature.npy')
wave1_v = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_valid_nature.npy')
wave1_te = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_test_nature.npy')

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[6,:,:])
train_wave1_q75 = scaler.transform(wave1_tr[6,:,:])
valid_wave1_q75 = scaler.transform(wave1_v[6,:,:])
test_wave1_q75 = scaler.transform(wave1_te[6,:,:])

X_train = train_wave1_q75
Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

X_valid =  valid_wave1_q75
Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

X_test = test_wave1_q75
Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []
for i in range(5): #experiment 5 times
  print("Round ",i+1)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  #y = GlobalAveragePooling1D(data_format='channels_last')(input_this)
  #y = Dense(512,activation='relu')(y)
  y = Dense(256,activation='relu')(input_this)
  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')

  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)
  preds = model.predict(X_test)

  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)

print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))



2layer MLP

In [ ]:
wave1_tr = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_nature.npy')
wave1_v = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_valid_nature.npy')
wave1_te = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_test_nature.npy')

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[6,:,:])
train_wave1_q75 = scaler.transform(wave1_tr[6,:,:])
valid_wave1_q75 = scaler.transform(wave1_v[6,:,:])
test_wave1_q75 = scaler.transform(wave1_te[6,:,:])

X_train = train_wave1_q75
Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

X_valid =  valid_wave1_q75
Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

X_test = test_wave1_q75
Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []
for i in range(5):
  print("Round ",i+1)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  #y = GlobalAveragePooling1D(data_format='channels_last')(input_this)
  y = Dense(256,activation='relu')(input_this)
  y = Dense(128,activation='relu')(y)
  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')

  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)
  preds = model.predict(X_test)

  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)

print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))



3layer MLP

In [ ]:
wave1_tr = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_nature.npy')
wave1_v = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_valid_nature.npy')
wave1_te = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_test_nature.npy')

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[6,:,:])
train_wave1_q75 = scaler.transform(wave1_tr[6,:,:])
valid_wave1_q75 = scaler.transform(wave1_v[6,:,:])
test_wave1_q75 = scaler.transform(wave1_te[6,:,:])

X_train = train_wave1_q75
Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

X_valid =  valid_wave1_q75
Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

X_test = test_wave1_q75
Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []
for i in range(5):
  print("Round ",i+1)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  #y = GlobalAveragePooling1D(data_format='channels_last')(input_this)
  y = Dense(256,activation='relu')(input_this)
  y = Dense(128,activation='relu')(y)
  y = Dense(64,activation='relu')(y)
  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')

  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)
  preds = model.predict(X_test)

  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)

print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))



very small MLP

In [ ]:
wave1_tr = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_nature.npy')
wave1_v = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_valid_nature.npy')
wave1_te = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_test_nature.npy')

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[6,:,:])
train_wave1_q75 = scaler.transform(wave1_tr[6,:,:])
valid_wave1_q75 = scaler.transform(wave1_v[6,:,:])
test_wave1_q75 = scaler.transform(wave1_te[6,:,:])

X_train = train_wave1_q75
Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

X_valid =  valid_wave1_q75
Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

X_test = test_wave1_q75
Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []
for i in range(5):
  print("Round ",i+1)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  #y = GlobalAveragePooling1D(data_format='channels_last')(input_this)
  y = Dense(32,activation='relu')(input_this)

  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')

  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)
  preds = model.predict(X_test)

  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)

print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))



## Flatness *

In [ ]:
atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_nature.npz")
# npz file saved with rms, cs, bw, fl, roll
# acoustic features are 1x431 dimensional
X_train = atr['arr_3']

atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_valid_nature.npz")
X_valid = atr['arr_3']


atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_test_nature.npz")
X_test = atr['arr_3']


Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

scaler = StandardScaler(with_mean=True, with_std=True).fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []

for i in range(5):
  print('Round ',i)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  y = Dense(256,activation='relu')(input_this)
  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')
  model.summary()

  # training
  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)

  #  evaluating
  preds = model.predict(X_test)

  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)
  del model
print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))
print(acc_perclass)

2layer MLP
avg and std: 32.32$\pm$2.57\\

In [ ]:
atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_nature.npz")
# npz file saved with rms, cs, bw, fl, roll
# acoustic features are 1x431 dimensional
X_train = atr['arr_3']

atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_valid_nature.npz")
X_valid = atr['arr_3']


atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_test_nature.npz")
X_test = atr['arr_3']


Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

scaler = StandardScaler(with_mean=True, with_std=True).fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []

for i in range(5):
  print('Round ',i)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  y = Dense(256,activation='relu')(input_this)
  y = Dense(128,activation='relu')(y)
  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')
  model.summary()

  # training
  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)

  #  evaluating
  preds = model.predict(X_test)

  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)
  del model
print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))


3layer MLP

34.74$\pm$2.72

In [ ]:
atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_nature.npz")
# npz file saved with rms, cs, bw, fl, roll
# acoustic features are 1x431 dimensional
X_train = atr['arr_3']

atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_valid_nature.npz")
X_valid = atr['arr_3']


atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_test_nature.npz")
X_test = atr['arr_3']


Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

scaler = StandardScaler(with_mean=True, with_std=True).fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []

for i in range(5):
  print('Round ',i)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  y = Dense(256,activation='relu')(input_this)
  y = Dense(128,activation='relu')(y)
  y = Dense(64,activation='relu')(y)
  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')
  model.summary()

  # training
  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)

  #  evaluating
  preds = model.predict(X_test)

  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)
  del model
print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))


very small MLP

In [ ]:
atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_train_nature.npz")
# npz file saved with rms, cs, bw, fl, roll
# acoustic features are 1x431 dimensional
X_train = atr['arr_3']

atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_valid_nature.npz")
X_valid = atr['arr_3']


atr = np.load("/content/drive/My Drive/Colab Notebooks/sardbscene_xai/acoustic_test_nature.npz")
X_test = atr['arr_3']


Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

scaler = StandardScaler(with_mean=True, with_std=True).fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']
acc = []
acc_perclass = []
for i in range(5):
  print('Round ',i)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  y = Dense(32,activation='relu')(input_this)
  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')
  model.summary()

  # training
  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)

  #  evaluating
  preds = model.predict(X_test)

  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)
  del model
print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))
print(acc_perclass)

## 25th quantile

In [ ]:
wave1_tr = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_nature.npy')
wave1_v = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_valid_nature.npy')
wave1_te = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_test_nature.npy')

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[5,:,:])
train_wave1_q75 = scaler.transform(wave1_tr[5,:,:])
valid_wave1_q75 = scaler.transform(wave1_v[5,:,:])
test_wave1_q75 = scaler.transform(wave1_te[5,:,:])

X_train = train_wave1_q75
Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

X_valid =  valid_wave1_q75
Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

X_test = test_wave1_q75
Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []
for i in range(5):
  print("Round ",i+1)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  #y = GlobalAveragePooling1D(data_format='channels_last')(input_this)
  #y = Dense(512,activation='relu')(y)
  y = Dense(256,activation='relu')(input_this)
  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')
  model.summary()
  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)
  preds = model.predict(X_test)
  del model
  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)

print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))



2layer MLP

In [ ]:
wave1_tr = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_nature.npy')
wave1_v = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_valid_nature.npy')
wave1_te = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_test_nature.npy')

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[5,:,:])
train_wave1_q75 = scaler.transform(wave1_tr[5,:,:])
valid_wave1_q75 = scaler.transform(wave1_v[5,:,:])
test_wave1_q75 = scaler.transform(wave1_te[5,:,:])

X_train = train_wave1_q75
Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

X_valid =  valid_wave1_q75
Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

X_test = test_wave1_q75
Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []
for i in range(5):
  print("Round ",i+1)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  #y = GlobalAveragePooling1D(data_format='channels_last')(input_this)
  y = Dense(256,activation='relu')(input_this)
  y = Dense(128,activation='relu')(y)
  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')
  model.summary()
  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)
  preds = model.predict(X_test)
  del model
  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)

print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))



3layer MLP

In [ ]:
wave1_tr = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_nature.npy')
wave1_v = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_valid_nature.npy')
wave1_te = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_test_nature.npy')

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[5,:,:])
train_wave1_q75 = scaler.transform(wave1_tr[5,:,:])
valid_wave1_q75 = scaler.transform(wave1_v[5,:,:])
test_wave1_q75 = scaler.transform(wave1_te[5,:,:])

X_train = train_wave1_q75
Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

X_valid =  valid_wave1_q75
Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

X_test = test_wave1_q75
Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []
for i in range(5):
  print("Round ",i+1)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  #y = GlobalAveragePooling1D(data_format='channels_last')(input_this)
  y = Dense(256,activation='relu')(input_this)
  y = Dense(128,activation='relu')(y)
  y = Dense(64,activation='relu')(y)
  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')
  model.summary()
  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)
  preds = model.predict(X_test)
  del model
  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)

print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))



very small MLP

In [ ]:
wave1_tr = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_train_nature.npy')
wave1_v = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_valid_nature.npy')
wave1_te = np.load('/content/drive/My Drive/Colab Notebooks/sardbscene_xai/addwavelets1_test_nature.npy')

scaler = StandardScaler(with_mean=True, with_std=True).fit(wave1_tr[5,:,:])
train_wave1_q75 = scaler.transform(wave1_tr[5,:,:])
valid_wave1_q75 = scaler.transform(wave1_v[5,:,:])
test_wave1_q75 = scaler.transform(wave1_te[5,:,:])

X_train = train_wave1_q75
Y_train = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/train_nature_sc_labels.npy')
train_sc_labelcat = keras.utils.to_categorical(Y_train-1)

X_valid =  valid_wave1_q75
Y_valid = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/valid_nature_sc_labels.npy')
valid_sc_labelcat = keras.utils.to_categorical(Y_valid-1)

X_test = test_wave1_q75
Y_test = np.load('/content/drive/My Drive/Colab Notebooks/SARdBScene_features/test_nature_sc_labels.npy')
test_sc_labelcat = keras.utils.to_categorical(Y_test-1)

numClasses=12
class_names = ['1','2','3','4','5','6','7','8','9','10','11','12']

acc = []
acc_perclass = []
for i in range(5):
  print("Round ",i+1)
  input_this = Input(shape=(X_train[0].shape))
  print(input_this)

  #y = GlobalAveragePooling1D(data_format='channels_last')(input_this)
  y = Dense(32,activation='relu')(input_this)

  out = Dense(numClasses, activation='softmax')(y) #for source counting

  model = Model(input_this,out)

  model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics='acc')
  model.summary()
  es = EarlyStopping(monitor='val_loss',verbose=1,patience=10,restore_best_weights=True)
  history = model.fit(X_train, train_sc_labelcat, validation_data=(X_valid,valid_sc_labelcat),
                    batch_size=64, epochs=100, callbacks=[es])

  model_plot(history)
  preds = model.predict(X_test)
  del model
  results(preds, test_sc_labelcat,class_names)

  a, b = pa1(preds,test_sc_labelcat,numClasses)

  acc.append(a)
  acc_perclass.append(b)

print("PA1 avg and std ", np.mean(acc),"±", np.std(acc))

